In [1]:
import pandas as pd
import numpy as np


application_train = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/application_train.csv')
pos_cash= pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/POS_CASH_balance.csv')
bureau_balance = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/bureau_balance.csv')
previous_application = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/previous_application.csv')
install_payments = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/installments_payments.csv')
credit_card = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/credit_card_balance.csv')
bureau = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/bureau.csv')
application_test = pd.read_csv('/home/sidharthjindal/HV/home-credit-default-risk/application_test.csv')


In [2]:
#these categorical variables hold binary data, so could be converted to 1 ,0
application_train = application_train[application_train['CODE_GENDER'].notna()]
col_for_dummies=application_train.select_dtypes(include=['O']).columns.drop(['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','EMERGENCYSTATE_MODE'])

In [3]:
application_train_dummies = pd.get_dummies(application_train, columns = col_for_dummies, drop_first = True)
application_test_dummies = pd.get_dummies(application_test, columns = col_for_dummies, drop_first = True)

In [4]:
application_train_dummies['CODE_GENDER'] = application_train_dummies['CODE_GENDER'].map( {'F':1, 'M':0})
application_train_dummies['FLAG_OWN_CAR'] = application_train_dummies['FLAG_OWN_CAR'].map( {'Y':1, 'N':0})
application_train_dummies['FLAG_OWN_REALTY'] = application_train_dummies['FLAG_OWN_REALTY'].map( {'Y':1, 'N':0})
application_train_dummies['EMERGENCYSTATE_MODE'] = application_train_dummies['EMERGENCYSTATE_MODE'].map( {'Yes':1, 'No':0})

application_test_dummies['CODE_GENDER'] = application_test_dummies['CODE_GENDER'].map({'F':1, 'M':0})
application_test_dummies['FLAG_OWN_CAR'] = application_test_dummies['FLAG_OWN_CAR'].map( {'Y':1, 'N':0})
application_test_dummies['FLAG_OWN_REALTY'] = application_test_dummies['FLAG_OWN_REALTY'].map( {'Y':1, 'N':0})
application_test_dummies['EMERGENCYSTATE_MODE'] = application_test_dummies['EMERGENCYSTATE_MODE'].map( {'Yes':1, 'No':0})


train_labels = application_train_dummies['TARGET']
# Align the training and testing data, keep only columns present in both dataframes
application_train_dummies, application_test_dummies = application_train_dummies.align(application_test_dummies, join = 'inner', axis = 1)
# Add the target back in
application_train_dummies['TARGET'] = train_labels


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
application_train_dummies['Percent_Days_employed'] = application_train_dummies['DAYS_EMPLOYED']/application_train_dummies['DAYS_BIRTH']*100
application_train_dummies['Credit_as_percent_income'] = application_train_dummies['AMT_CREDIT']/application_train_dummies['AMT_INCOME_TOTAL']*100
application_train_dummies['Credit_flag'] = application_train_dummies['AMT_INCOME_TOTAL'] > application_train_dummies['AMT_CREDIT']
application_train_dummies['Annuity_as_percent_income'] = application_train_dummies['AMT_ANNUITY']/ application_train_dummies['AMT_INCOME_TOTAL']*100
application_train_dummies['Credit_to_annuity'] = application_train_dummies['AMT_CREDIT']/ application_train_dummies['AMT_ANNUITY']
application_train_dummies['Credit_to_Goods'] = application_train_dummies['AMT_CREDIT']/ application_train_dummies['AMT_GOODS_PRICE']


application_test_dummies['Percent_Days_employed'] = application_test_dummies['DAYS_EMPLOYED']/application_test_dummies['DAYS_BIRTH']*100
application_test_dummies['Credit_as_percent_income'] = application_test_dummies['AMT_CREDIT']/application_test_dummies['AMT_INCOME_TOTAL']*100
application_test_dummies['Credit_flag'] = application_test_dummies['AMT_INCOME_TOTAL'] > application_test_dummies['AMT_CREDIT']
application_test_dummies['Annuity_as_percent_income'] = application_test_dummies['AMT_ANNUITY']/ application_test_dummies['AMT_INCOME_TOTAL']*100
application_test_dummies['Credit_to_annuity'] = application_test_dummies['AMT_CREDIT']/ application_test_dummies['AMT_ANNUITY']
application_test_dummies['Credit_to_Goods'] = application_test_dummies['AMT_CREDIT']/ application_test_dummies['AMT_GOODS_PRICE']

In [6]:
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]

application_bureau = application_train_dummies.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau[grp.columns].fillna(0))

application_bureau_test = application_test_dummies.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test[grp.columns].fillna(0))

In [7]:
#max days since credit ended
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR'])['DAYS_ENDDATE_FACT'].max().reset_index().rename(columns = {'DAYS_ENDDATE_FACT': 'DAYS_ENDDATE_FACT_MAX'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau['DAYS_ENDDATE_FACT_MAX'].fillna(0))

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test['DAYS_ENDDATE_FACT_MAX'].fillna(0))

In [8]:
#MAX_Annuity_Approved to Current_Annuity Ratio
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR'])['AMT_ANNUITY'].max().reset_index().rename(columns = {'AMT_ANNUITY': 'AMT_ANNUITY_MAX'})

grp2 = application_bureau[['SK_ID_CURR','AMT_ANNUITY']]
grp2 = grp2.merge(grp, on = 'SK_ID_CURR', how = 'left')
grp2.update(grp2['AMT_ANNUITY_MAX'].fillna(0))
grp2['CURRENT_ANNUITY_TO_AMT_ANNUITY_MAX'] = grp2['AMT_ANNUITY']/grp2['AMT_ANNUITY_MAX']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del(grp2['AMT_ANNUITY'])
del(grp2['AMT_ANNUITY_MAX'])
application_bureau = application_bureau.merge(grp2, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau['CURRENT_ANNUITY_TO_AMT_ANNUITY_MAX'].fillna(0))

grp2 = application_bureau_test[['SK_ID_CURR','AMT_ANNUITY']]
grp2 = grp2.merge(grp, on = 'SK_ID_CURR', how = 'left')
grp2.update(grp2['AMT_ANNUITY_MAX'].fillna(0))
grp2['CURRENT_ANNUITY_TO_AMT_ANNUITY_MAX'] = grp2['AMT_ANNUITY']/grp2['AMT_ANNUITY_MAX']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del(grp2['AMT_ANNUITY'])
del(grp2['AMT_ANNUITY_MAX'])
application_bureau_test = application_bureau_test.merge(grp2, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test['CURRENT_ANNUITY_TO_AMT_ANNUITY_MAX'].fillna(0))



#Mean_Annuity_Approved to Current_Annuity Ratio
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR'])['AMT_ANNUITY'].mean().reset_index().rename(columns = {'AMT_ANNUITY': 'AMT_ANNUITY_MEAN'})

grp2 = application_bureau[['SK_ID_CURR','AMT_ANNUITY']]
grp2 = grp2.merge(grp, on = 'SK_ID_CURR', how = 'left')
grp2.update(grp2['AMT_ANNUITY_MEAN'].fillna(0))
grp2['CURRENT_ANNUITY_TO_AMT_ANNUITY_MEAN'] = grp2['AMT_ANNUITY']/grp2['AMT_ANNUITY_MEAN']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del(grp2['AMT_ANNUITY'])
del(grp2['AMT_ANNUITY_MEAN'])
application_bureau = application_bureau.merge(grp2, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau['CURRENT_ANNUITY_TO_AMT_ANNUITY_MEAN'].fillna(0))

grp2 = application_bureau_test[['SK_ID_CURR','AMT_ANNUITY']]
grp2 = grp2.merge(grp, on = 'SK_ID_CURR', how = 'left')
grp2.update(grp2['AMT_ANNUITY_MEAN'].fillna(0))
grp2['CURRENT_ANNUITY_TO_AMT_ANNUITY_MEAN'] = grp2['AMT_ANNUITY']/grp2['AMT_ANNUITY_MEAN']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del(grp2['AMT_ANNUITY'])
del(grp2['AMT_ANNUITY_MEAN'])
application_bureau_test = application_bureau_test.merge(grp2, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test['CURRENT_ANNUITY_TO_AMT_ANNUITY_MEAN'].fillna(0))

In [9]:
#Ratio of credit requested to max credit repaid previous time
grp = bureau[bureau['CREDIT_ACTIVE']=="Closed"].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].max().reset_index().rename(columns = {'AMT_CREDIT_SUM': 'AMT_CREDIT_SUM_MAX_CLOSED'})

grp2 = application_bureau[['SK_ID_CURR','AMT_CREDIT']]
grp2 = grp2.merge(grp,on='SK_ID_CURR',how='left')
grp2 = grp2.fillna(0)
grp2['AmountCredit_OVER_MaxAmountCredited_Closed'] = grp2['AMT_CREDIT']/grp2['AMT_CREDIT_SUM_MAX_CLOSED']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del grp2['AMT_CREDIT']
del grp2['AMT_CREDIT_SUM_MAX_CLOSED']
application_bureau = application_bureau.merge(grp2, on='SK_ID_CURR', how='left')


grp2 = application_bureau_test[['SK_ID_CURR','AMT_CREDIT']]
grp2 = grp2.merge(grp,on='SK_ID_CURR',how='left')
grp2 = grp2.fillna(0)
grp2['AmountCredit_OVER_MaxAmountCredited_Closed'] = grp2['AMT_CREDIT']/grp2['AMT_CREDIT_SUM_MAX_CLOSED']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del grp2['AMT_CREDIT']
del grp2['AMT_CREDIT_SUM_MAX_CLOSED']
application_bureau_test = application_bureau_test.merge(grp2, on='SK_ID_CURR', how='left')





grp = bureau[bureau['CREDIT_ACTIVE']=="Active"].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].max().reset_index().rename(columns = {'AMT_CREDIT_SUM': 'AMT_CREDIT_SUM_MAX_ACTIVE'})

grp2 = application_bureau[['SK_ID_CURR','AMT_CREDIT']]
grp2 = grp2.merge(grp,on='SK_ID_CURR',how='left')
grp2 = grp2.fillna(0)
grp2['AmountCredit_OVER_MaxAmountCredited_Active'] = grp2['AMT_CREDIT']/grp2['AMT_CREDIT_SUM_MAX_ACTIVE']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del grp2['AMT_CREDIT']
del grp2['AMT_CREDIT_SUM_MAX_ACTIVE']
application_bureau = application_bureau.merge(grp2, on='SK_ID_CURR', how='left')


grp2 = application_bureau_test[['SK_ID_CURR','AMT_CREDIT']]
grp2 = grp2.merge(grp,on='SK_ID_CURR',how='left')
grp2 = grp2.fillna(0)
grp2['AmountCredit_OVER_MaxAmountCredited_Active'] = grp2['AMT_CREDIT']/grp2['AMT_CREDIT_SUM_MAX_ACTIVE']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del grp2['AMT_CREDIT']
del grp2['AMT_CREDIT_SUM_MAX_ACTIVE']
application_bureau_test = application_bureau_test.merge(grp2, on='SK_ID_CURR', how='left')

In [10]:
# Active Credit(Sum) to Income ratio
grp = bureau[bureau['CREDIT_ACTIVE']=="Active"].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(columns = {'AMT_CREDIT_SUM': 'AMT_CREDIT_SUM_SUM'})

grp2 = application_bureau[['SK_ID_CURR','AMT_INCOME_TOTAL']]
grp2 = grp2.merge(grp,on='SK_ID_CURR',how='left')
grp2 = grp2.fillna(0)
grp2['Active_Credit_to_Income_Ratio'] = grp2['AMT_CREDIT_SUM_SUM']/grp2['AMT_INCOME_TOTAL']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del grp2['AMT_INCOME_TOTAL']
del grp2['AMT_CREDIT_SUM_SUM']
application_bureau = application_bureau.merge(grp2, on='SK_ID_CURR', how='left')


grp2 = application_bureau_test[['SK_ID_CURR','AMT_INCOME_TOTAL']]
grp2 = grp2.merge(grp,on='SK_ID_CURR',how='left')
grp2 = grp2.fillna(0)
grp2['Active_Credit_to_Income_Ratio'] = grp2['AMT_CREDIT_SUM_SUM']/grp2['AMT_INCOME_TOTAL']
grp2 = grp2.replace([np.inf, -np.inf], 0)
del grp2['AMT_INCOME_TOTAL']
del grp2['AMT_CREDIT_SUM_SUM']
application_bureau_test = application_bureau_test.merge(grp2, on='SK_ID_CURR', how='left')


In [11]:
# Combining categorical features
bureau_categorical = pd.get_dummies(bureau.select_dtypes('object'))
bureau_categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']

grp = bureau_categorical.groupby(by = ['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau[grp.columns].fillna(0))

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test[grp.columns].fillna(0))

In [12]:
# Number of past loans per customer
grp = bureau[bureau['CREDIT_ACTIVE']=="Closed"].groupby(by = ['SK_ID_CURR'])['SK_ID_BUREAU'].count().reset_index().rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT_CLOSED'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau['BUREAU_LOAN_COUNT_CLOSED'] = application_bureau['BUREAU_LOAN_COUNT_CLOSED'].fillna(0)

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test['BUREAU_LOAN_COUNT_CLOSED'] = application_bureau_test['BUREAU_LOAN_COUNT_CLOSED'].fillna(0)


grp = bureau[bureau['CREDIT_ACTIVE']=="Active"].groupby(by = ['SK_ID_CURR'])['SK_ID_BUREAU'].count().reset_index().rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT_ACTIVE'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau['BUREAU_LOAN_COUNT_ACTIVE'] = application_bureau['BUREAU_LOAN_COUNT_ACTIVE'].fillna(0)

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test['BUREAU_LOAN_COUNT_ACTIVE'] = application_bureau_test['BUREAU_LOAN_COUNT_ACTIVE'].fillna(0)

In [13]:
# Number of types of past loans per customer 
grp = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau['BUREAU_LOAN_TYPES'] = application_bureau['BUREAU_LOAN_TYPES'].fillna(0)

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test['BUREAU_LOAN_TYPES'] = application_bureau_test['BUREAU_LOAN_TYPES'].fillna(0)

In [14]:
# Debt over credit ratio 
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM': 'TOTAL_CREDIT_SUM'})

grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CREDIT_SUM_DEBT'})

grp1['DEBT_CREDIT_RATIO'] = grp2['TOTAL_CREDIT_SUM_DEBT']/grp1['TOTAL_CREDIT_SUM']

del grp1['TOTAL_CREDIT_SUM']

application_bureau = application_bureau.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau['DEBT_CREDIT_RATIO'].head()
application_bureau['DEBT_CREDIT_RATIO'] = application_bureau['DEBT_CREDIT_RATIO'].fillna(0)
application_bureau['DEBT_CREDIT_RATIO'] = application_bureau.replace([np.inf, -np.inf], 0)
application_bureau['DEBT_CREDIT_RATIO'] = pd.to_numeric(application_bureau['DEBT_CREDIT_RATIO'], downcast='float')

application_bureau_test = application_bureau_test.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau_test['DEBT_CREDIT_RATIO'] = application_bureau_test['DEBT_CREDIT_RATIO'].fillna(0)
application_bureau_test['DEBT_CREDIT_RATIO'] = application_bureau_test.replace([np.inf, -np.inf], 0)
application_bureau_test['DEBT_CREDIT_RATIO'] = pd.to_numeric(application_bureau_test['DEBT_CREDIT_RATIO'], downcast='float')


In [15]:
# Overdue over debt ratio
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CUSTOMER_DEBT'})

grp1['OVERDUE_DEBT_RATIO'] = grp1['TOTAL_CUSTOMER_OVERDUE']/grp2['TOTAL_CUSTOMER_DEBT']


del grp1['TOTAL_CUSTOMER_OVERDUE']

application_bureau = application_bureau.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau['OVERDUE_DEBT_RATIO'] = application_bureau['OVERDUE_DEBT_RATIO'].fillna(0)
application_bureau['OVERDUE_DEBT_RATIO'] = application_bureau.replace([np.inf, -np.inf], 0)
application_bureau['OVERDUE_DEBT_RATIO'] = pd.to_numeric(application_bureau['OVERDUE_DEBT_RATIO'], downcast='float')

application_bureau_test = application_bureau_test.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau_test['OVERDUE_DEBT_RATIO'] = application_bureau_test['OVERDUE_DEBT_RATIO'].fillna(0)
application_bureau_test['OVERDUE_DEBT_RATIO'] = application_bureau_test.replace([np.inf, -np.inf], 0)
application_bureau_test['OVERDUE_DEBT_RATIO'] = pd.to_numeric(application_bureau_test['OVERDUE_DEBT_RATIO'], downcast='float')


In [16]:
# Number of previous applications per customer
grp = previous_application[['SK_ID_CURR','SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV':'PREV_APP_COUNT'})

# Take only the IDs which are present in application_bureau
application_bureau_prev = application_bureau.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test = application_bureau_test.merge(grp, on =['SK_ID_CURR'], how = 'left')

#Fill NA for previous application counts (lets say there was an application ID present in application_bureau but not present
# in grp, then that means that person never took loan previously, so count of previous loan for that person = 0)
application_bureau_prev['PREV_APP_COUNT'] = application_bureau_prev['PREV_APP_COUNT'].fillna(0)
application_bureau_prev_test['PREV_APP_COUNT'] = application_bureau_prev_test['PREV_APP_COUNT'].fillna(0)

In [17]:
# Combining numerical features

#Take the mean of all the parameters (grouping by SK_ID_CURR)
grp = previous_application.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()

#Add prefix prev in front of all columns so that we know that these columns are from previous_application
prev_columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]

#Change the columns
grp.columns = prev_columns

application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))
application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [18]:
# Combining categorical features
prev_categorical = pd.get_dummies(previous_application.select_dtypes('object'))
prev_categorical['SK_ID_CURR'] = previous_application['SK_ID_CURR']

grp = prev_categorical.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]

application_bureau_prev = application_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [19]:
# Combining numerical features
grp = pos_cash.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns

In [20]:
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))


In [21]:
# Combining categorical features
pos_cash_categorical = pd.get_dummies(pos_cash.select_dtypes('object'))
pos_cash_categorical['SK_ID_CURR'] = pos_cash['SK_ID_CURR']

grp = pos_cash_categorical.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]

In [22]:
application_bureau_prev = application_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [23]:
# Combining numerical features and there are no categorical features in this dataset
grp = install_payments.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['INSTALL_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))
application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [24]:
#last_loan_late_payment (mean and max)
grp = install_payments[['SK_ID_CURR','DAYS_ENTRY_PAYMENT','DAYS_INSTALMENT']]

grp['late_payment'] = grp['DAYS_ENTRY_PAYMENT'] - grp['DAYS_INSTALMENT']

del(grp['DAYS_ENTRY_PAYMENT'])
del(grp['DAYS_INSTALMENT'])

grp2 = grp.groupby(by =['SK_ID_CURR']).mean().reset_index().rename(columns={'late_payment':'late_payment_mean'})
grp3 = grp.groupby(by =['SK_ID_CURR']).max().reset_index().rename(columns={'late_payment':'late_payment_max'})

application_bureau_prev = application_bureau_prev.merge(grp2, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev = application_bureau_prev.merge(grp3, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev['late_payment_max'].fillna(0))
application_bureau_prev.update(application_bureau_prev['late_payment_mean'].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp2, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test = application_bureau_prev_test.merge(grp3, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test['late_payment_max'].fillna(0))
application_bureau_prev_test.update(application_bureau_prev_test['late_payment_mean'].fillna(0))

/home/sidharthjindal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [25]:

# Combining numerical features
grp = credit_card.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [26]:
# Combining categorical features
credit_categorical = pd.get_dummies(credit_card.select_dtypes('object'))
credit_categorical['SK_ID_CURR'] = credit_card['SK_ID_CURR']

grp = credit_categorical.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]

application_bureau_prev = application_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [27]:
for col in application_bureau_prev:
    if("/" in col):
        application_bureau_prev.rename(columns = {col : col[:col.find("/")]}, inplace = True)
    elif(": " in col):
         application_bureau_prev.rename(columns = {col : col.replace(": ","_")}, inplace = True)
    elif(":" in col):
         application_bureau_prev.rename(columns = {col : col.replace(":","_")}, inplace = True)
            
for col in application_bureau_prev_test:
    if("/" in col):
        application_bureau_prev_test.rename(columns = {col : col[:col.find("/")]}, inplace = True)
    elif(": " in col):
         application_bureau_prev_test.rename(columns = {col : col.replace(": ","_")}, inplace = True)
    elif(":" in col):
         application_bureau_prev_test.rename(columns = {col : col.replace(":","_")}, inplace = True)

In [28]:
pd.set_option('display.max_columns', True)
pd.set_option('display.max_rows', True)
print(len(application_bureau_prev.columns))
print(len(application_bureau_prev_test.columns))
# for i in application_bureau_prev.columns:
#     print(i)

491
490


In [29]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
import random
# X=application_bureau_prev.drop(columns=['TARGET'], axis=1)
# y=application_bureau_prev['TARGET']


In [43]:
# X=application_bureau_prev.dropna()
X=application_bureau_prev.fillna(0)
y=X['TARGET']
X=X.drop(columns=['TARGET'], axis=1)
print(X.shape)
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

embeded_rf_selector = SelectFromModel(RandomForestClassifier(random_state = 0, n_jobs=-1, class_weight="balanced"
,bootstrap= True,
 max_depth= 20,
 min_samples_leaf= 5,
 min_samples_split= 40,
 n_estimators= 200,
max_features = 'sqrt')
                                      
, threshold='6*median')
embeded_rf_selector.fit(X, y)

(307511, 490)


SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight='balanced',
                                                 criterion='gini', max_depth=20,
                                                 max_features='sqrt',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=5,
                                                 min_samples_split=40,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=200, n_jobs=-1,
                                                 oob_score=False,
                                             

In [44]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

74 selected features


In [45]:
embeded_rf_feature

['SK_ID_CURR',
 'CODE_GENDER',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'HOUR_APPR_PROCESS_START',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'TOTALAREA_MODE',
 'DAYS_LAST_PHONE_CHANGE',
 'NAME_EDUCATION_TYPE_Higher education',
 'Percent_Days_employed',
 'Credit_as_percent_income',
 'Annuity_as_percent_income',
 'Credit_to_annuity',
 'Credit_to_Goods',
 'BUREAU_DAYS_CREDIT',
 'BUREAU_DAYS_CREDIT_ENDDATE',
 'BUREAU_DAYS_ENDDATE_FACT',
 'BUREAU_AMT_CREDIT_MAX_OVERDUE',
 'BUREAU_AMT_CREDIT_SUM',
 'BUREAU_AMT_CREDIT_SUM_DEBT',
 'BUREAU_DAYS_CREDIT_UPDATE',
 'DAYS_ENDDATE_FACT_MAX',
 'AmountCredit_OVER_MaxAmountCredited_Closed',
 'AmountCredit_OVER_MaxAmountCredited_Active',
 'Active_Credit_to_Income_Ratio',
 'BUREAU_CREDIT_ACTIVE_Active',
 'BUREAU_CREDIT_ACTIVE_Closed',
 'BUREAU_LOAN_COUNT_ACTIVE',
 'DEBT_CREDIT_RATIO',
 'OVERDUE_DEBT_RATIO',
 '

In [46]:
X_imputation_train = application_bureau_prev[embeded_rf_feature]


X_imputation_test = application_bureau_prev_test[embeded_rf_feature]

In [47]:
imputer = IterativeImputer(BayesianRidge())

imputed_total_train = pd.DataFrame(imputer.fit_transform(X_imputation_train))
imputed_total_train.columns = X_imputation_train.columns

imputed_total_test = pd.DataFrame(imputer.fit_transform(X_imputation_test))
imputed_total_test.columns = X_imputation_test.columns

In [48]:
# tweaking the hyper parameters-score 0.73789
y=application_bureau_prev[['SK_ID_CURR','TARGET']]

X = imputed_total_train.merge(y, on='SK_ID_CURR',how ='left')
#all rows with all important columns and target

y1=X['TARGET']
X1=X[X_imputation_train.columns.drop('SK_ID_CURR')]

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=1)

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state = 0, n_jobs=-1, class_weight="balanced"
,bootstrap= True,
 max_depth= 20,
 min_samples_leaf= 5,
 min_samples_split= 40,
 n_estimators= 200,
max_features = 'sqrt')
rf_clf.fit(X_train, y_train)
print('Accuracy on Training set:',cross_val_score(rf_clf, X_train, y_train, cv=3, scoring='accuracy'))
print('Accuracy on Test set:',cross_val_score(rf_clf, X_test, y_test, cv=3, scoring='accuracy'))

Accuracy on Training set: [0.89707694 0.89632087 0.8958562 ]
Accuracy on Test set: [0.90156578 0.90590703 0.90346812]


In [49]:
import gc
gc.collect()
imputed_total_test[X_imputation_test.columns.drop(['SK_ID_CURR'])].isna().sum().sum()

0

In [50]:
predictions = rf_clf.predict_proba(imputed_total_test[X_imputation_test.columns.drop(['SK_ID_CURR'])])

In [51]:
result = []
for i in predictions:
    result.append(i[1])
# predictions = [i if (i > j) else j for [i,j] in predictions]
# np.unique(predictions,return_counts = True)

In [52]:
np.unique(result,return_counts = True)


(array([0.01219221, 0.01754679, 0.0178011 , ..., 0.81951688, 0.83715112,
        0.88035004]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [53]:
predictions_df=pd.DataFrame(data={"SK_ID_CURR":application_bureau_prev_test["SK_ID_CURR"],"TARGET":result}) 

In [54]:
predictions_df[['SK_ID_CURR','TARGET']].to_csv("Result.csv")

In [55]:
predictions_df.shape

(48744, 2)